# Lunar Lander Notebook
This notebook demonstrates GNP landing a spacecraft.


# Small Tutorial Using Fracnetics

## Summary 🔭

This notebook demonstrates how to use the **Fracnetics** library to solve the CartPole environment problem from Gymnasium (fork from OpenAI Gym).

Fractnetics is a Python library for **Genetic Network Programming (GNP)** enhanced with fractal geometry, which means it a subfield of Evolutionary Algorithms.

## Lunar Lander

The Lunar Lander problem is a classic reinforcement learning task where an agent must safely land a spacecraft on a designated landing pad. The lander can control its main and side thrusters to adjust its position and velocity. The goal is to land softly and upright, while avoiding crashes or going out of bounds. Rewards are given for touching down gently and penalties for collisions, fuel use, or leaving the landing area. This problem tests an agent’s ability to balance precision control with strategic planning under continuous dynamics.

![Beschreibung](https://gymnasium.farama.org/_images/lunar_lander.gif)

## Evolutionary Aglortihms 🧬

Evolutionary Algorithms are a family of optimization techniques inspired by the process of natural evolution. They work by maintaining a **population** of candidate solutions, which evolve over time through operations such as **selection, mutation, and crossover**. The idea is to iteratively improve solutions by mimicking survival of the fittest, where better solutions are more likely to be chosen and combined to form new ones. EAs are especially useful for solving complex, nonlinear, and high-dimensional problems where traditional optimization methods may fail.

## Genetic Network Programming 🦾

Genetic Network Programming is a branch of evolutionary computation that represents solutions as **networks** of nodes rather than as linear strings (like in Genetic Algorithms). Each node corresponds to a function, decision, or action, and the network’s structure allows for recurrent flows of control. This enables GNP to model adaptive, flexible, and memory-dependent behaviors. It has been applied successfully in areas such as robotics, decision-making systems, and dynamic optimization tasks.

## Next 🔥
In this tutorial we will initialize a population, train it, validate it, and finally record a run.  

Checkout Fracnetics and Gymnasium here:

- Fracnetics: https://github.com/FabianKoehnke/fracnetics
- Gymnasium: https://gymnasium.farama.org/



## Install and Load Packages

In [1]:
!pip install --upgrade fracnetics
!pip install --upgrade pyvis
import fracnetics as fn
import gymnasium as gym
from matplotlib import pyplot as plt
import statistics
from gymnasium.wrappers import RecordVideo
from pyvis.network import Network # for network visualization 
from IPython.display import HTML # for network visualization


[notice] A new release of pip is available: 25.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


## Initializing the Population

Here we initialize the population for Fracnetics to solve the cartpole problem.

Key parameters:
- `seed`: random seed for reproducibility  
- `ni`: number of individuals  
- `jn` / `jnf`: judgment nodes and functions  
- `pn` / `pnf`: perceptron nodes and functions  
- `fractalJudgment`: enables/disables fractal-based judgment (not relevant in this tutorial)

In [2]:
seed=17
# Initialize the population
pop = fn.Population(
    seed=seed,
    ni=1000,       # number of individuals
    jn=1,         # judgment nodes
    jnf=8,        # judgment node functions
    pn=2,         # perceptron nodes
    pnf=4,        # perceptron node functions
    fractalJudgment=False
)

Hints:

- We just initialized one jn and two pn because networks can grow and shrink and therefore can add judgment nodes during the evolution. See also our paper: https://link.springer.com/chapter/10.1007/978-3-031-90062-4_18
- We initialized 4 judgment nodes functions (jnf) because the observation is a ndarray with shape (4,)
- We initializes 2 processing node functions (2) because the action is a ndarray with shape (1,) which can take values {0, 1}


In [3]:
# Set input feature boundaries (based on CartPole state space)
minFeatures = [ -2.5, -2.5, -10, -10, -6.2831855, -10, -0, -0 ] 
maxFeatures = [ 2.5, 2.5, 10, 10, 6.2831855, 10, 1, 1 ]
pop.setAllNodeBoundaries(minFeatures, maxFeatures)

## Training the Population

The population is trained over 300 generations.  
Each generation involves:
1. Fitness evaluation in the Gym environment  
2. Selection (Tournament Selection)  
3. Mutation (edges)  
4. Crossover  
5. Adding/Deleting nodes  

The best fitness score from each generation is stored and plotted.  


In [ ]:
# Training environment
env = gym.make("LunarLander-v3")
fitnessProgess = []

nTimes200 = 0
maxFitness = 0
g = 0
while maxFitness < 200 or nTimes200 <= 10:
    pop.gymnasium(
        env,
        dMax=10,
        maxSteps=1000,
        maxConsecutiveP=10,
        worstFitness=0,
        seed=seed+g
    )
    # Selection
    pop.tournamentSelection(
        N=5, # tournament size
        E=1 # number of safed elite
        )
    
    # Add/Delete nodes
    pop.callAddDelNodes(minFeatures, maxFeatures)

    # Mutations
    pop.callEdgeMutation(
        probInnerNodes = 0.05, # probability of changing an edge of jn or pn
        probStartNode = 0.05 # probability of changing an edge of the start node
        )

    pop.callBoundaryMutationEdgeSizeDependingSigma(0.05,1)
    # Crossover (recombination)
    pop.crossover(probability=0.05)

    maxFitness = pop.bestFit
    if maxFitness >= 200:
        nTimes200 += 1
    else:
        nTimes200 = 0

    g += 1
    print(f"Generation: {g} | Maximal Fitness: {maxFitness}")
    fitnessProgess.append([ind.fitness for ind in pop.individuals]) # append fitness of each individual for boxplot chart

Generation: 1 | Maximal Fitness: 20.27112579345703
Generation: 2 | Maximal Fitness: 0.0
Generation: 3 | Maximal Fitness: 0.0
Generation: 4 | Maximal Fitness: 20.94660186767578
Generation: 5 | Maximal Fitness: 16.802833557128906
Generation: 6 | Maximal Fitness: 11.565284729003906
Generation: 7 | Maximal Fitness: 47.186737060546875
Generation: 8 | Maximal Fitness: 0.0
Generation: 9 | Maximal Fitness: 26.342010498046875
Generation: 10 | Maximal Fitness: 0.0
Generation: 11 | Maximal Fitness: 19.640113830566406
Generation: 12 | Maximal Fitness: 6.2520294189453125
Generation: 13 | Maximal Fitness: 32.81517028808594
Generation: 14 | Maximal Fitness: 0.0
Generation: 15 | Maximal Fitness: 0.0
Generation: 16 | Maximal Fitness: 19.77825164794922
Generation: 17 | Maximal Fitness: 17.20281219482422
Generation: 18 | Maximal Fitness: 25.340377807617188
Generation: 19 | Maximal Fitness: 0.0
Generation: 20 | Maximal Fitness: 0.0
Generation: 21 | Maximal Fitness: 0.0
Generation: 22 | Maximal Fitness: 9.

In [ ]:
# Plot fitness progression
plt.figure(figsize=(15, 6))
plt.boxplot(fitnessProgess, whis=2, sym=".")
plt.title("Fitness Progress")
plt.xlabel("Generation")
plt.ylabel("Fitness")
ticks = range(0,len(fitnessProgess), 10)
plt.xticks(ticks,labels = ticks)
plt.show()

In [ ]:
ticks = range(1, 100 + 1, 5)
labels = [f"Label {i}" for i in ticks]
print(labels)

## Inspecting the Best Individual

Now we inspect the best individual:  
- Start node and its edges  
- Inner node types, functions, edges, and boundaries  

Because the Genetic Network Programming is **not** a Blackbox-Model


In [ ]:
pop.individuals[-1].fitness
print(f"Start Node: {pop.individuals[-1].startNode.edges}")
for node in pop.individuals[-1].innerNodes:
  print(f"Type: {node.type} | Function: {node.f} Edges: {node.edges} | Boundaries: {node.boundaries}")

We can also visualize the network in a plot, for example using the pyvis package.

Colors:

- Start node =
- Processing nodes =
- Judgment nodes =

Tip: You can view the boundaries of an edge by hovering over it with your mouse.

In [ ]:
net = Network(notebook=True, directed=True, cdn_resources="in_line")
net.force_atlas_2based()
individual = pop.individuals[-1]
processingFunctionNames = ["do nothing", "fire left orientation engine", "fire main engine", "fire right orientation engine"]
judgmentFunctionNames = ["x", "y", "linear velocities x", "linear velocities y", "angle", "angular velocity", "left contact", "right contact"]

# adding start node
net.add_node(-1, label=individual.startNode.type, color = "#635b3e")

# adding inner nodes
for node in individual.innerNodes:
    # set color
    if node.type == "J":
        color = "#3e6341"
        net.add_node(node.id, label=f"F: {judgmentFunctionNames[node.f]}", color=color)
    elif node.type == "P":
        color = "#372f61"
        net.add_node(node.id, label=f"F: {processingFunctionNames[node.f]}", color=color)
    else:
        color = None
        
for node in individual.innerNodes:
    for idx, edge in enumerate(node.edges):
        if node.type == "J":
            edgeLabel = f"{node.boundaries[idx]} bis {node.boundaries[idx+1]}"
            net.add_edge(node.id, edge, title = edgeLabel,
                    font={'size': 14, 'color': '#3e6341', 'align': 'horizontal'})
        else:
            net.add_edge(node.id, edge,
                    font={'size': 14, 'color': '#372f61', 'align': 'horizontal'})
        
# adding start node edge 
net.add_edge(-1, individual.startNode.edges[0])
net.save_graph("cartpole_solution.html")
HTML(filename="cartpole_solution.html")

## Validation of the Best Individual

The best individual is validated in a fresh environment.  
We compute the average fitness across multiple runs.  


In [ ]:
env = gym.make("CartPole-v1")
validationResults = []
for v in range(10):
    pop.gymnasium(
          env,
          dMax=10,
          maxSteps=500,
          maxConsecutiveP=10,
          worstFitness=0,
          seed=seed+v)
    validationResults.append(pop.bestFit)
print(f"Average Fitnes of Validations: {statistics.mean(validationResults)}")

## Rendering and Recording the Best Run

Finally, we run the environment in `rgb_array` mode and use `RecordVideo`  
to save a video of the best individual playing CartPole. You can find the video in the nodebook folder "videos".


In [ ]:
env = gym.make("CartPole-v1", render_mode="rgb_array")
env = RecordVideo(env, video_folder="videos", name_prefix="cartpole")

# Keep only the best individual
pop.individuals = [pop.individuals[-1]]

# Record a run
pop.gymnasium(
    env,
    dMax=10,
    maxSteps=500,
    maxConsecutiveP=5,
    worstFitness=0,
    seed=seed
)
env.close()